In [5]:
%autosave 0

Autosave disabled


In [6]:
import ipywidgets as widgets
import json
import time
import os
from IPython.display import display, HTML
from traitlets import Int, Unicode, observe

In [7]:
import requests
assert(requests.get('http://pshannon.systemsbiology.net/js/clustergrammer.js').status_code == 200)   

In [15]:
class CGRWidget(widgets.DOMWidget):

    _view_name = Unicode('CGRView').tag(sync=True)
    _view_module = Unicode('cgr').tag(sync=True)


In [13]:
%%javascript
"use strict"

require.config({

    paths: {'jquery'          : 'http://code.jquery.com/jquery-1.12.4.min',
            'jquery-ui'       : 'http://code.jquery.com/ui/1.12.1/jquery-ui.min',
            'jquery-dataTable': 'https://cdn.datatables.net/1.10.13/js/jquery.dataTables.min',
            'cytoscape'       : 'http://cytoscape.github.io/cytoscape.js/api/cytoscape.js-latest/cytoscape',
            'bootstrap'       : 'http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min',
            'igv'             : 'http://igv.org/web/release/1.0.7/igv-1.0.7',
            'three'           : 'https://cdnjs.cloudflare.com/ajax/libs/three.js/r83/three',
            'underscore'      : 'https://cdnjs.cloudflare.com/ajax/libs/underscore.js/1.8.3/underscore-min',
            'clustergrammer'  : 'http://pshannon.systemsbiology.net/js/clustergrammer',
            'app3d'           : 'http://pshannon.systemsbiology.net/js/app3d',
            'd3'              : 'https://cdnjs.cloudflare.com/ajax/libs/d3/4.7.3/d3',
            },
    shim: {'bootstrap':   {'deps'   : ['jquery']},
           'igv':         {'deps'   : ['jquery', 'jquery-ui', 'bootstrap']},
           'three':       {'exports': 'THREE'},
           'underscore':  {'exports': '_'},
           }
    });

require.undef('cgrDemo')

define('cgrDemo', ["jupyter-js-widgets", "jquery", "jquery-ui", "jquery-dataTable", "cytoscape", "igv", 
                   'three', 'app3d', 'd3', 'clustergrammer'], 
    function(widgets, $, ui, DataTable, cytoscape, igv, THREE, app3d, d3, clustergrammer) {
    
    var CGRView = widgets.DOMWidgetView.extend({

        initialize: function() {
          console.log("CGRView.initialize")
          },

        render: function() {
            var cgrDiv =  $("<div id='heatmapDiv' style='border:1px solid gray; height: 800px; width: 97%'></div>");
            this.$el.append(this.masterTabsDiv); 
            createHeatMap();
           },

        createHeatmap: function() {
          var network_data =
            {"views":
               [{"N_row_sum": "all", "dist": "cos", "nodes":
                  {"col_nodes": [{"ini": 3, "rank": 2, "name": "s01_120405", "clust": 1,
                                  "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 2},
                                 {"ini": 2, "rank": 1, "name": "s02_120405", "clust": 0,
                                  "group": [3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                                 {"ini": 1, "rank": 0, "name": "s03_120405", "clust": 2,
                                  "group": [2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 1}],
                   "row_nodes": [{"ini": 2, "rank": 0, "name": "HLTF", "clust": 0,
                                  "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                                 {"ini": 1, "rank": 1, "name": "POU2F1", "clust": 1,
                                  "group": [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0], "rankvar": 1}]}},
                {"dist": "cos", "N_row_var": "all", "nodes":
                  {"col_nodes": [{"ini": 3, "rank": 2, "name": "s01_120405", "clust": 1,
                                  "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 2},
                                 {"ini": 2, "rank": 1, "name": "s02_120405", "clust": 0,
                                  "group": [3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                                 {"ini": 1, "rank": 0, "name": "s03_120405", "clust": 2,
                                  "group": [2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 1}],
                   "row_nodes": [{"ini": 2, "rank": 0, "name": "HLTF", "clust": 0,
                                  "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                                 {"ini": 1, "rank": 1, "name": "POU2F1", "clust": 1,
                                  "group": [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0], "rankvar": 1}]}}],
               "col_nodes": [{"ini": 3, "rank": 2, "name": "s01_120405", "clust": 1,
                              "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 2},
                             {"ini": 2, "rank": 1, "name": "s02_120405", "clust": 0,
                              "group": [3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                             {"ini": 1, "rank": 0, "name": "s03_120405", "clust": 2,
                              "group": [2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 1}],
               "links": [{"target": 0, "value": 5.591991, "source": 0},
                         {"target": 1, "value": 11.939007, "source": 0},
                         {"target": 2, "value": 7.738552, "source": 0},
                         {"target": 0, "value": 31.060965999999997, "source": 1},
                         {"target": 1, "value": 18.00348, "source": 1},
                         {"target": 2, "value": 21.577569, "source": 1}],
               "row_nodes": [{"ini": 2, "rank": 0, "name": "HLTF", "clust": 0,
                              "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                             {"ini": 1, "rank": 1, "name": "POU2F1", "clust": 1,
                              "group": [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0], "rankvar": 1}]};
       
          make_clust(network_data);
          var about_string = 'Zoom, scroll, and click buttons to interact with the clustergram. <a href="http://amp.pharm.mssm.edu/clustergrammer/help"> <i class="fa fa-question-circle" aria-hidden="true"></i> </a>';
          function make_clust(network_data){
             var args = {
                'root': '#heatmapDiv',
                'network_data': network_data,
                'about': about_string,
                'sidebar_width':150,
                };
             var screen_width = window.innerWidth;
             var screen_height = window.innerHeight - 20;
             $("#heatmapDiv").width(screen_width);
             $("#heatmapDiv").height(screen_height);
             cgm = Clustergrammer(args);
             $("#heatmapDiv .wait_message").remove()
             console.log('loading clustergrammer')
             } // make_clust
         }, // createHeatmap
        
    });
    return{CGRView: CGRView}
    });
 

<IPython.core.display.Javascript object>

In [12]:
cgr = CGRWidget()